<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
import re
import nltk
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
import pandas as pd
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [3]:
try:
    comments = pd.read_csv(r'C:\Users\nknt\OneDrive - Chevron\Downloads\toxic_comments.csv', index_col = 0)
except:
    comments = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0) 

comments

In [4]:
def clear_text(text):
    re_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(re_text.split())

In [5]:
comments['clean_text'] = comments['text'].apply(clear_text)

In [6]:
comments['text'][0]

In [7]:
sentence = comments['clean_text'][0]
sentence

In [8]:
word_list = nltk.word_tokenize(sentence)
print(word_list)

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Совет 🤔:



Стоит убрать лишний код.  Ниже У тебя есть импорт WordNetLemmatizer, tqdm,  функция lemmed.  Зачем же тогда эта ячейка?


In [10]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
%%time

from nltk import WordNetLemmatizer
from tqdm import tqdm

wnl = WordNetLemmatizer()

def lemmed(text):
    result = list(wnl.lemmatize(t, get_wordnet_pos(t)) for t in nltk.word_tokenize(text.lower()))
    return result


lemmed_text = lemmed(sentence)

assert len(word_list) == len(lemmed_text)

print(lemmed_text)

In [12]:
comments = comments.reset_index(drop=True)
comments

In [13]:
%%time

def lemm_df(df):
    result = []
    for i in tqdm(range(len(df))):
        row = df.iloc[i]
        result.append(lemmed(row['clean_text']))
    df['lemm_text'] = pd.Series(result)
lemm_df(comments)

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



Очистка или лемматизация проведена,  yо стоило в обоих случаях использовать apply, в пандас циклы неэффективны




</div>


In [14]:
comments

In [15]:
del comments['text']
del comments['clean_text']
comments

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:
    
Мне пришлось подправить твой код чтобы запустилось. Для некоторых особо строгих ревьюеров это основание завернуть работу без проверки, так что будь внимательней в следующий раз )
    
Совет - используй конструкцию try except:    
  
    try:

        commentsdata = pd.read_csv('C:/your_project/users_behavior.csv')
    except:


        comments = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)    

    
    
<div class="alert alert-warning">

Совет: 


Если не знаешь - чтобы не было столбца  `Unnamed: 0` при чтении файла можно так:


    pd.read_csv(..., index_col=0)

    
(`Unnamed: 0` появляется при не совсем корректном сохранении файла)    

<div>    

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:

    
- стоит провести очистку текста - убираем цифры, знаки препинания, слова не на латинском


    
- нет лемматизации, а без него у нас одни и те же слова стоящие в разных падежах и временах будут расцениваться как разные, в итоге колличество фичей раздувается, а качество прогноза будет падать. 


Набираемся ума-разума [тут](https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/) ) . Там примеры разных библиотек для лемматизации, выбирай на свой вкус ). Рекомендую WordNetLemmatizer (но не забыть о POS тег, используй функцию `get_wordnet_pos` из примера и обрати внимание на то что WordNetLemmatizer на лемматизацию принимает не предложение, а слово) или [SpaCy](https://spacy.io/).  

<div class="alert alert-warning">


Совет: 
    



- после очистки и лематизации (и убрав стопслова) можно провести частотный анализ текста/[облако слов](https://habr.com/ru/post/517410/) - чтобы получить общее представление о тематике и о наиболее часто встерчаемых словах Кроме того графики, рисунки делают проект визуально интересней
    
    

In [3]:
#corpus = comments['text'].values.astype('U')
#corpus

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:



Да, в тренажере был текст на кирилице, там перевод в unicode оправдан. В нашем случае (латиница) это лишь  увеличит количество потребляемой памяти и это в лучшем случаи, в худшем он обрушает ядро.





</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



Убрал .values.astype('U')




</div>


In [16]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

In [17]:
corpus = map(' '.join, comments['lemm_text'])

In [18]:
train_x, test_x, train_y, test_y = train_test_split(pd.Series(list(corpus)), comments['toxic'], random_state=42)

In [19]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df=0.001) 
train_x = count_tf_idf.fit_transform(train_x)
test_x = count_tf_idf.transform(test_x)
print("Размер train матрицы:", train_x.shape)
print("Размер test матрицы:", test_x.shape)




<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:

Обучаем векторайз на тренировочном датасете, а затем обученный применяем к тестовому. Кстати почему?


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



Теперь верно.  А делаем так чтобы не было утечки информации, ведь по логике моделирования у нас есть информация только о тренировочной выборке




</div>


## Обучение

In [20]:
%%time

model_1 = LogisticRegression(random_state=42)
model_1.fit(train_x,train_y)

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:


Не забываем при инициализации модели о random_state, иначе после каждого запуска кода у нас может быть разный результат


</div>


In [21]:
train_preds = model_1.predict(train_x)
print('F1 score for Logistic Regression train set is ',f1_score(train_preds,train_y))

In [23]:
%%time

model_2 = RandomForestClassifier(random_state=42)
model_2.fit(train_x,train_y)

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:


test датасет используем в самом конце, когда тестируем лучшую модель. А пока не выберем лучшую модель на валидации, тестовой выборки для нас как будто не существует. Так что тебе нужна валидационная выборка,  или везде использовать GridSearchCV, который сам её создаёт внутри себя, симметрику по ней можно посмотреть через .best_score_ 


Напомню о логике  использования датасетов:
    
1. На train мы обучаем
2. По валидации смотрим на результаты обучения (следим чтобы не было переобучения и/или делаем подбор гиперпараметров).  И выбираем лучшую модель. 
3. Тестовая (out-of_sample) у нас для финальной проверки, когда определена модель с конкретными гиперпараметрами.

In [24]:
train_preds2 = model_2.predict(train_x)
print('F1 score for RF Classifier train set is ',f1_score(train_preds2,train_y))

От данной модели отказываемся, так как скорее всего она переобучена.

In [25]:
%%time

model_3 = LGBMClassifier(random_state=42)
model_3.fit(train_x,train_y)

In [26]:
train_preds3 = model_3.predict(train_x)
print('F1 score for Light GB Machine Classifier is ',f1_score(train_preds3,train_y))

In [73]:
# тестирование лучшей модели
test_preds3 = model_3.predict(test_x)
print('F1 score for Light GB Machine Classifier Classifier test set is ',f1_score(test_preds3,test_y))

Так как данной меры F-1 недостаточно по требованию заказчика (миниимум 0.75) переобучаем модель с помощью тюнинга гиперпараметров.

In [70]:
%%time

parameters = {
     'n_estimators': [500, 1000],
     'learning_rate':[0.05],
     'num_leaves':[50, 100]
}

model_4 = GridSearchCV(LGBMClassifier(random_state=42), cv=3, param_grid = parameters, n_jobs=-1, verbose=10, scoring='f1')
model_4.fit(train_x,train_y)

# лучшее значение F1 на кросс-валидации
print(f'F1 score for Light GB Machine Classifier with HP Tuning on train set is: {model_4.best_score_}')

In [72]:
test_preds4 = model_4.predict(test_x)
print('F1 score for Light GB Machine Classifier with HP Tuning on test set is ',f1_score(test_preds4,test_y))

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех: 

- Теперь логика моделирования не нарушена
    
    
    
    

<div class="alert alert-warning">



Совет: 


Что может помочь добиться лучшего результата (от простого)? 




- можно поиграться [порогом](https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/)
   

- учесть дисбаланс классов в таргете. (но не oversampling, это скользкая дорожка, через class_weight) Но порой это  приводит к ухудшению результата


    




    
 - полезно настраивать векторайзеры и избегать утечки данных при использовании GS и предобработки данных  (тут пригодится pipeline)


     

- сгенерировать новые фичи, например  например посчитать число слов в тексте, длину слов итп итд. Или с помощью [тематического моделирования](https://pythobyte.com/python-for-nlp-topic-modeling-8fb3d689/) 
    
    
    


- использование предбученной модели Берта, выбрав соответствующую модель и используя полученные эмбединги, даже на небольшом тренировочном датасете можно обучить модель, которая на test покажет хорошую метрику. В этом случаи можно сразу получить метрику > 0.95 (при правильно выбранной модели)

</div>


## Выводы

Данные по токсичным комментариям были успешно загружены, очищены от ненужных символов, и лематизированны. Далее были объединены в корпус, и стоп-слова были удалены. Далее, данные были разделены на обучающую и тестовую выборки. В конце они были векторизованы с помощью TF-IDF и конвертированы в матрицу. Были обучены 4 модели - логистическая регрессия, случайный лес, градиентный бустинг и последний но с подбором гиперпараметров. Последняя модель была создана, так как градиентный бустинг показал самый близкий показатель меры f1 - 0.73 при требовании 0.75. Лучшая модель показала результат - 0.75, что соответствует требованиям заказчика.


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>




Спасибо за работу Нурлан, но в настоящий момент я не могу ее принять, так что мы с тобой еще пообщаемся )




Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить).
    



Обязательное к исправлению:



- неверный путь к файлу



- нет очистки и Лематизации
    
    


- .astype('U') лишнее, стоит экономить ресурсы, иначе может даже ядро обрушиться



- на train делаем .fit векторайз, на test .transform



- при инициализации модели не забываем random_state (можно один раз вначале просто прописать random.seed(42), чтобы не прописывать каждый раз везде random_state. Кстати знаешь откуда 42?)




- на test датасете тестируем только лучшую модель (нарушена логика использования датасетов при моделировании)



    
    
Жду исправлений, для принятия проекта. Если какие то вопросы, то сразу спрашивай ) 


</div>



<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Спасибо за работу! 
    
    
    

Красного нет, вопросов нет, значит все, пора принимать) Надеюсь мои советы и вопросики были полезны и в копилочку знаний упало что то новое, а проект стал лучше, и симпатичней. Желаю успехов дальнейшей учебе Нурлан




<font color='green'><b>Полезные (и просто интересные) материалы:</b> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 


Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .

Если нравится смотреть и слушать то есть целый курс на Ютубе https://www.youtube.com/watch?v=qDMwIQRQt-M&list=PLEwK9wdS5g0qksxWxtE5c2KuFkIfUXe3i&index=1

</font>


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны

<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>

Привет Нурлан! Меня зовут Марат, и я буду твоим ревьюером. Спешу сообщить что все ключевые этапы в работе выполнены,  с задачей тебе удалось справиться. По поводу обращения - в IT сфере принято общаться на «ты» :) Но, если привычней на «вы», дай знать. Как ревьюера моя задача помочь тебе в развитии, дав хорошие советы. Я внимательно посмотрю твой код, ознакомлюсь с твоими выводами и оставлю комментарии. Где то могу предложить небольшие исправление в коде, но ненавязчиво. Где потребуются уточнения, я оставлю много наводящих вопросов. Они помогут тебя с поиском верного решения.

Все мои комментарии размечены по цветам, для лучшего восприятия. 
    
<div class="alert alert-success">Зеленым цветом и словом «Успех» отмечены особо удачные и элегантные решения, которыми ты можешь гордиться. </div>
        
<div class="alert alert-warning">Желтым и значком словом «Совет», помечены решения у которых есть альтернативные решения, более оптимальные. Ты можешь найти их сразу и доработать проект, или отложить это на потом, для будущих проектах. Проект будет принят и без их доработки. </div>
        
<div class="alert alert-danger"> Красным цветом и значком словом «Ошибка» помечу твои решения, на которые стоит обратить внимание прежде всего. После их доработки проект будет принят. </div>
        
Залог успеха - работа сообща, взаимное уважение и работа в диалоге. Поэтому, помечай свои ответные комментарии на мои реплики заметным цветом или курсивом, так мне будет легче их отслеживать. Пожалуйста, не изменяй и не удаляй мои комментарии. Все это поможет выполнить повторную проверку быстрей.

</div>